 ### Smiles Retrieval Using PubChem API


In [3]:
import pandas as pd
import pubchempy as pcp
from tqdm import tqdm
import numpy as np

##### Load DrugComb Dataset

In [4]:
drug_comb=pd.read_csv("drug_interaction_drugcomb.csv",low_memory=False)

In [5]:
drug_comb.head()

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
0,1,5-FU,ABT-888,A2058,uM,uM,30.869,3.865915,6.256584,-2.951386,5.536903,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.839
1,2,5-FU,ABT-888,A2058,uM,uM,27.460,8.247403,12.333896,3.125927,11.614215,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.430
2,3,5-FU,ABT-888,A2058,uM,uM,29.901,6.063440,11.660209,2.452239,10.940528,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.871
3,4,5-FU,ABT-888,A2058,uM,uM,24.016,-4.280231,5.145209,-4.062761,4.425528,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.986
4,5,5-FU,AZD1775,A2058,uM,uM,66.847,12.284698,15.765467,10.409407,18.656340,5.126836,0.266027,11.471,25.164,76.501,57.193,30.212


In [6]:
drug_comb.shape

(1048575, 18)

In [7]:
drug_comb['drug_row'].isna().sum()  #Counts the number of missing (NaN) values in the 'drug_row' column 

np.int64(0)

In [8]:
drug_comb['drug_col'].isna().sum()  #Counts the number of missing (NaN) values in the 'drug_col' column 

np.int64(582542)

##### Missing drug names in the drug_col column were not imputed, as such data is considered sensitive and unsuitable for imputation. Instead, rows containing missing values in drug_col were removed to ensure the integrity of downstream processing.


In [9]:
drug_comb = drug_comb.dropna(subset='drug_col')  # Drops all rows from the DataFrame where the 'drug_col' entry is missing (NaN)

In [10]:
drug_comb['drug_col'].isna().sum()

np.int64(0)

In [13]:
a=len(np.unique(np.concatenate([drug_comb.drug_row.unique(),drug_comb.drug_col.unique()])))
print(f"total number of drug involved in interaciton dataset {a}")

total number of drug involved in interaciton dataset 4150


In [14]:
drug_comb.shape

(466033, 18)

In [18]:
drug_comb[drug_comb['drug_row'].isna() | drug_comb['drug_col'].isna()] # Selects all rows from drug_comb where either 'drug_row' or 'drug_col' has missing (NaN) values

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S


##### The distinct drug names are compiled into a new DataFrame called drug_df to facilitate further analysis, such as retrieving SMILES strings

In [17]:
unique_drugs = pd.unique(drug_comb[['drug_row', 'drug_col']].values.ravel())
drug_df = pd.DataFrame(unique_drugs, columns=['drug'])
drug_df.head()

,drug
0,5-FU
1,ABT-888
2,AZD1775
3,BEZ-235
4,BORTEZOMIB


In [18]:
drug_df.shape

(4150, 1)

##### Fetching SMILES Strings from PubChem by Drug Name

In [20]:
import requests

failed_names = []

def get_smiles_from_name(name):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/property/IsomericSMILES/TXT'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text.strip()
    except Exception as e:
        print(f"Error retrieving SMILES for {name}: {e}")
        failed_names.append(name)
        return None


##### Adding a new 'smiles' column to drug_df by applying the get_smiles_from_name function to each drug name, fetching its SMILES string from PubChem

In [21]:
# Enabling progress_apply to show progress bars with pandas apply
tqdm.pandas()
drug_df['smiles'] = drug_df['drug'].progress_apply(get_smiles_from_name)

  2%|█▌                                                                              | 81/4150 [01:13<58:54,  1.15it/s]

Error retrieving SMILES for (+/-)-SULFINPYRAZONE: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(+/-)-SULFINPYRAZONE/property/IsomericSMILES/TXT


 10%|███████▌                                                                       | 395/4150 [06:03<55:53,  1.12it/s]

Error retrieving SMILES for (6-)E-<U+200B>AMINOCAPROIC ACID: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(6-)E-%3CU+200B%3EAMINOCAPROIC%20ACID/property/IsomericSMILES/TXT


 14%|███████████                                                                    | 584/4150 [08:56<54:35,  1.09it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████▌| 4129/4150 [1:02:29<00:18,  1.15it/s]

Error retrieving SMILES for AC1Q4OOX: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/AC1Q4OOX/property/IsomericSMILES/TXT


100%|███████████████████████████████████████████████████████████████████████████▊| 4142/4150 [1:02:40<00:07,  1.07it/s]

Error retrieving SMILES for Vincristine Sulfate, Apocynaceae sp.: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Vincristine%20Sulfate,%20Apocynaceae%20sp./property/IsomericSMILES/TXT


100%|████████████████████████████████████████████████████████████████████████████| 4150/4150 [1:02:49<00:00,  1.10it/s]


##### Save the drug_df DataFrame with drug names and their SMILES strings to a CSV file


In [22]:
drug_df.to_csv("final_drugname_with_smiles")